In [2]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset as HFDataset
import os
import json
from sklearn.model_selection import train_test_split
import warnings
import re
warnings.filterwarnings('ignore')

In [3]:
# ==================== Configuration ====================
class Config:
    # Model settings
    MODEL_NAME = "t5-base"
    MAX_INPUT_LENGTH = 512
    MAX_TARGET_LENGTH = 512  # Increased for full SOAP notes

    # Training settings
    BATCH_SIZE = 4
    LEARNING_RATE = 3e-5
    NUM_EPOCHS = 8
    WARMUP_STEPS = 500
    WEIGHT_DECAY = 0.01

    # Paths
    DATA_PATH = "/content/mtsamples.csv"
    MODEL_OUTPUT_DIR = "models/soap_generator"
    CHECKPOINT_DIR = "checkpoints"

    # Device
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# ==================== Data Preparation ====================
class SOAPDataset(Dataset):
    """Custom Dataset for SOAP note generation"""

    def __init__(self, data, tokenizer, max_input_len, max_target_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]

        # Create input with task prefix
        input_text = f"summarize to SOAP format: {item['input']}"
        target_text = item['target']

        # Tokenize
        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_input_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_target_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': target_encoding['input_ids'].squeeze()
        }


In [5]:
# ==================== SOAP Note Parser ====================
def extract_soap_sections(text):
    """
    Extract SOAP sections from medical transcription.
    Returns dict with subjective, objective, assessment, plan sections.
    """
    text = text.strip()

    sections = {
        'subjective': '',
        'objective': '',
        'assessment': '',
        'plan': ''
    }

    # Common patterns for SOAP sections
    patterns = {
        'subjective': r'SUBJECTIVE:?\s*(.*?)(?=OBJECTIVE:|PHYSICAL EXAMINATION:|ASSESSMENT:|$)',
        'objective': r'(?:OBJECTIVE:|PHYSICAL EXAMINATION:)\s*(.*?)(?=ASSESSMENT:|IMPRESSION:|PLAN:|$)',
        'assessment': r'(?:ASSESSMENT:|IMPRESSION:)\s*(.*?)(?=PLAN:|RECOMMENDATIONS:|$)',
        'plan': r'(?:PLAN:|RECOMMENDATIONS:)\s*(.*?)$'
    }

    for section, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        if match:
            sections[section] = match.group(1).strip()

    return sections

def is_soap_formatted(text):
    """Check if text contains SOAP format sections"""
    soap_keywords = ['SUBJECTIVE:', 'OBJECTIVE:', 'ASSESSMENT:', 'PLAN:']
    count = sum(1 for keyword in soap_keywords if keyword in text.upper())
    return count >= 2  # At least 2 SOAP sections present

def create_condensed_input(transcription, description):
    """
    Create a condensed input from full transcription.
    This simulates raw clinical notes that need to be converted to SOAP.
    """
    # Remove SOAP headers to create "raw" input
    condensed = transcription

    # Remove explicit SOAP section markers
    condensed = re.sub(r'SUBJECTIVE:?\s*', '', condensed, flags=re.IGNORECASE)
    condensed = re.sub(r'OBJECTIVE:?\s*', '', condensed, flags=re.IGNORECASE)
    condensed = re.sub(r'ASSESSMENT:?\s*', '', condensed, flags=re.IGNORECASE)
    condensed = re.sub(r'IMPRESSION:?\s*', '', condensed, flags=re.IGNORECASE)
    condensed = re.sub(r'PLAN:?\s*', '', condensed, flags=re.IGNORECASE)

    # Combine with description
    if description and len(description) > 10:
        condensed = f"{description}. {condensed}"

    # Clean up excessive whitespace
    condensed = re.sub(r'\s+', ' ', condensed).strip()

    return condensed


In [6]:
# ==================== Data Processing ====================
def load_and_process_data(config):
    """Load and process mtsamples.csv data"""

    print("📊 Loading mtsamples dataset...")

    try:
        # Load CSV
        data = pd.read_csv(config.DATA_PATH)
        print(f"📋 Original columns: {data.columns.tolist()}")
        print(f"📊 Original shape: {data.shape}")

        # Check if required columns exist
        if 'transcription' not in data.columns:
            raise ValueError("Column 'transcription' not found in CSV")

        # Remove rows with missing transcriptions
        data = data.dropna(subset=['transcription'])
        print(f"✅ After removing NaN: {len(data)} samples")

        # Process each row
        processed_data = []

        for idx, row in data.iterrows():
            transcription = str(row['transcription']).strip()
            description = str(row['description']) if 'description' in row and pd.notna(row['description']) else ''

            # Skip very short transcriptions
            if len(transcription) < 100:
                continue

            # Check if transcription is already in SOAP format
            if is_soap_formatted(transcription):
                # Create condensed input (remove SOAP markers)
                input_text = create_condensed_input(transcription, description)

                # Keep original as target (with SOAP format)
                target_text = transcription

                processed_data.append({
                    'input': input_text[:2000],  # Limit input length
                    'target': target_text[:3000]  # Limit target length
                })
            else:
                # If not in SOAP format, use description as input and full text as target
                # (though most mtsamples data is already in SOAP format)
                if description and len(description) > 20:
                    processed_data.append({
                        'input': description,
                        'target': transcription[:3000]
                    })

        # Convert to DataFrame
        processed_df = pd.DataFrame(processed_data)

        print(f"✅ Processed {len(processed_df)} valid SOAP note samples")

        if len(processed_df) < 10:
            print("⚠️ Too few samples. Check your CSV file format.")
            return None

        # Show sample
        print("\n📋 Sample processed data:")
        print(f"Input preview: {processed_df.iloc[0]['input'][:200]}...")
        print(f"Target preview: {processed_df.iloc[0]['target'][:200]}...")

        return processed_df

    except FileNotFoundError:
        print(f"❌ File not found: {config.DATA_PATH}")
        print("Please ensure the CSV file is in the correct location.")
        return None
    except Exception as e:
        print(f"❌ Error loading data: {str(e)}")
        return None


In [7]:
# ==================== Training ====================
def train_model(config):
    """Main training function"""

    print("🚀 Starting SOAP Note Generator Training")
    print(f"📱 Device: {config.DEVICE}")
    print(f"🤖 Model: {config.MODEL_NAME}")

    # Load data
    data = load_and_process_data(config)

    if data is None or len(data) < 10:
        print("❌ Cannot proceed with training due to insufficient data.")
        return None, None

    # Split data
    train_df, val_df = train_test_split(data, test_size=0.15, random_state=42)
    print(f"📚 Train samples: {len(train_df)}, Validation samples: {len(val_df)}")

    # Load tokenizer and model
    print("🔧 Loading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
    model = T5ForConditionalGeneration.from_pretrained(config.MODEL_NAME)

    # Create datasets
    train_dataset = SOAPDataset(train_df, tokenizer,
                                config.MAX_INPUT_LENGTH,
                                config.MAX_TARGET_LENGTH)
    val_dataset = SOAPDataset(val_df, tokenizer,
                             config.MAX_INPUT_LENGTH,
                             config.MAX_TARGET_LENGTH)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=config.CHECKPOINT_DIR,
        num_train_epochs=config.NUM_EPOCHS,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        learning_rate=config.LEARNING_RATE,
        weight_decay=config.WEIGHT_DECAY,
        warmup_steps=config.WARMUP_STEPS,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_dir='./logs',
        logging_steps=50,
        save_total_limit=2,
        fp16=torch.cuda.is_available(),
        report_to="none",
        gradient_accumulation_steps=2  # Helps with memory
    )

    # Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator
    )

    # Train
    print("🎓 Starting training...")
    try:
        trainer.train()
    except Exception as e:
        print(f"❌ Training error: {str(e)}")
        return None, None

    # Save model
    print(f"💾 Saving model to {config.MODEL_OUTPUT_DIR}")
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    trainer.save_model(config.MODEL_OUTPUT_DIR)
    tokenizer.save_pretrained(config.MODEL_OUTPUT_DIR)

    # Save configuration
    config_dict = {
        'model_name': config.MODEL_NAME,
        'max_input_length': config.MAX_INPUT_LENGTH,
        'max_target_length': config.MAX_TARGET_LENGTH,
    }
    with open(os.path.join(config.MODEL_OUTPUT_DIR, 'config.json'), 'w') as f:
        json.dump(config_dict, f)

    print("✅ Training complete!")
    print(f"📁 Model saved to: {config.MODEL_OUTPUT_DIR}")

    return model, tokenizer

In [8]:
# ==================== Testing ====================
def test_model(model, tokenizer, config):
    """Test the trained model"""

    print("\n🧪 Testing model...")

    test_cases = [
        "45-year-old male with chest pain, shortness of breath for 2 hours. BP 140/90, HR 95. ECG shows ST elevation in leads II, III, aVF. Patient anxious, diaphoretic. History of hypertension and smoking.",

        "28-year-old female presents with severe headache, photophobia, nausea for 24 hours. Temperature 101.2F, neck stiffness present. No recent trauma. Patient appears uncomfortable.",

        "65-year-old diabetic male with foot ulcer, right great toe, present for 3 weeks. Wound 2cm diameter, minimal drainage. Pedal pulses diminished. Patient denies fever."
    ]

    model.eval()

    for i, test_input in enumerate(test_cases, 1):
        print(f"\n{'='*60}")
        print(f"Test Case {i}:")
        print(f"{'='*60}")

        input_text = f"summarize to SOAP format: {test_input}"
        inputs = tokenizer(input_text, return_tensors="pt",
                          max_length=config.MAX_INPUT_LENGTH,
                          truncation=True).to(config.DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=config.MAX_TARGET_LENGTH,
                num_beams=4,
                early_stopping=True,
                temperature=0.7,
                do_sample=False,
                repetition_penalty=1.2,
                length_penalty=1.0
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"\n📝 Input:\n{test_input}")
        print(f"\n📋 Generated SOAP Note:\n{generated_text}")

In [ ]:
# ==================== Main ====================
if __name__ == "__main__":
    config = Config()

    # Check if data file exists
    if not os.path.exists(config.DATA_PATH):
        print(f"❌ Data file not found: {config.DATA_PATH}")
        print("Please upload mtsamples.csv to /content/ directory")
    else:
        model, tokenizer = train_model(config)

        if model is not None and tokenizer is not None:
            model = model.to(config.DEVICE)
            test_model(model, tokenizer, config)
            print("\n✨ All done! You can now use the model in the Streamlit app.")
        else:
            print("\n❌ Training failed. Please check the errors above.")